In [5]:
import os 
os.chdir("/Users/happyeon/Desktop/data_science/play_data")

In [6]:
import naver,dbconnect

In [7]:
help(dbconnect.connection)

Help on function connection in module dbconnect:

connection(host='127.0.0.1', user='happyeon', password='123', charset='utf8', db='play')
    host: temp
    return cur,con



In [8]:
cur,con = dbconnect.connection()

In [9]:
cur.execute("show tables")
cur.fetchall()

(('class3',), ('KRX',), ('stock_day',))

In [10]:
cur.execute("desc KRX")
cur.fetchall()


(('ISU_CD', 'varchar(200)', 'YES', '', None, ''),
 ('ISU_SRT_CD', 'varchar(200)', 'NO', 'PRI', None, ''),
 ('ISU_NM', 'varchar(200)', 'YES', '', None, ''),
 ('ISU_ABBRV', 'varchar(200)', 'YES', '', None, ''),
 ('ISU_ENG_NM', 'varchar(200)', 'YES', '', None, ''),
 ('LIST_DD', 'varchar(200)', 'YES', '', None, ''),
 ('MKT_TP_NM', 'varchar(200)', 'YES', '', None, ''),
 ('SECUGRP_NM', 'varchar(200)', 'YES', '', None, ''),
 ('SECT_TP_NM', 'varchar(200)', 'YES', '', None, ''),
 ('KIND_STKCERT_TP_NM', 'varchar(200)', 'YES', '', None, ''),
 ('PARVAL', 'varchar(200)', 'YES', '', None, ''),
 ('LIST_SHRS', 'varchar(200)', 'YES', '', None, ''))

In [11]:
import pandas as pd

In [12]:
df = pd.read_sql_query("select * from KRX",con)

/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [13]:
df[df['KIND_STKCERT_TP_NM'] == '보통주']['ISU_SRT_CD'].to_frame()

,ISU_SRT_CD
0,000020
1,000040
2,000050
3,000070
5,000080
...,...
2694,950170
2695,950190
2696,950200
2697,950210


In [15]:
from tqdm import tqdm
import numpy as np

sql = "INSERT INTO stock_day VALUES (%s , %s , %s , %s , %s , %s , %s , %s , %s)"
for code in tqdm(df[df['KIND_STKCERT_TP_NM'] == '보통주']['ISU_SRT_CD']):
    tmp_df = naver.get_stock(code,1)
    tmp_df[['종가', '전일비', '시가', '고가', '저가', '거래량']] = \
    tmp_df[['종가', '전일비', '시가', '고가', '저가', '거래량']].astype(np.int64)
    tmp_df['거래금액'] = tmp_df['종가'] * tmp_df['거래량']
    tmp_df['종목'] = code
    tmp_df = tmp_df[['종목', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '거래금액', ]]
    for idx, row in tmp_df.iterrows():
        try:
            cur.execute(sql,list(row.values))
        except Exception as e:
            print(code, e)
    
con.commit()


100%|██████████| 2577/2577 [02:49<00:00, 15.23it/s]


In [19]:
df2 = pd.read_sql_query("""SELECT * 
FROM stock_day sd  
WHERE `close`  = (SELECT MAX(`close`) FROM stock_day);""",con)

/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
df2

,symbol,stock_date,close,before,open,high,low,vol,money
0,003240,2023-02-21,820000,2000,831000,866000,817000,3981,3264420000


In [21]:
df = pd.read_sql_query("select * from KRX",con)

/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
df

,ISU_CD,ISU_SRT_CD,ISU_NM,ISU_ABBRV,ISU_ENG_NM,LIST_DD,MKT_TP_NM,SECUGRP_NM,SECT_TP_NM,KIND_STKCERT_TP_NM,PARVAL,LIST_SHRS
0,KR7000020008,000020,동화약품보통주,동화약품,DongwhaPharm,1976/03/24,KOSPI,주권,,보통주,"1,000","27,931,470"
1,KR7000040006,000040,KR모터스보통주,KR모터스,KR MOTORS,1976/05/25,KOSPI,주권,,보통주,500,"96,138,465"
2,KR7000050005,000050,경방보통주,경방,Kyungbang,1956/03/03,KOSPI,주권,,보통주,500,"27,415,270"
3,KR7000070003,000070,삼양홀딩스보통주,삼양홀딩스,SAMYANGHOLDINGS,1968/12/27,KOSPI,주권,,보통주,"5,000","8,564,271"
4,KR7000071001,000075,삼양홀딩스1우선주,삼양홀딩스우,SAMYANGHOLDINGS(1P),1992/02/21,KOSPI,주권,,구형우선주,"5,000","304,058"
...,...,...,...,...,...,...,...,...,...,...,...,...
2694,KR8392080006,950170,제이티씨,JTC,JTC Inc.,2018/04/06,KOSDAQ,주식예탁증권,관리종목(소속부없음),보통주,무액면,"50,093,024"
2695,KR8344390008,950190,미투젠,미투젠,ME2ZEN Limited,2020/08/18,KOSDAQ,주식예탁증권,외국기업(소속부없음),보통주,무액면,"13,579,892"
2696,KR8840150005,950200,소마젠,소마젠,"Psomagen, Inc.",2020/07/13,KOSDAQ,주식예탁증권,외국기업(소속부없음),보통주,0,"19,205,053"
2697,KR8702070002,950210,프레스티지바이오파마KDR,프레스티지바이오파마,PRESTIGE BIOPHARMA LIMITED KDR,2021/02/05,KOSPI,주식예탁증권,,보통주,무액면,"60,096,155"


In [24]:
df2 = pd.read_sql_query("select * from stock_day",con)

/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [25]:
df2

,symbol,stock_date,close,before,open,high,low,vol,money
0,000020,2023-02-09,9490,10,9500,9530,9440,79781,757121690
1,000020,2023-02-10,9430,60,9510,9520,9330,74148,699215640
2,000020,2023-02-13,9430,0,9430,9460,9360,92591,873133130
3,000020,2023-02-14,9400,30,9490,9520,9360,153609,1443924600
4,000020,2023-02-15,9270,130,9400,9410,9240,90373,837757710
...,...,...,...,...,...,...,...,...,...
25750,950220,2023-02-16,3840,25,3815,3875,3800,418284,1606210560
25751,950220,2023-02-17,3800,40,3800,3855,3790,385993,1466773400
25752,950220,2023-02-20,3815,15,3800,3825,3745,378151,1442646065
25753,950220,2023-02-21,3790,25,3795,3855,3790,447202,1694895580


In [39]:
stock = pd.read_sql_query("select * from stock_day",con)

/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [41]:
stock.groupby(['symbol'])['close'].agg(['max','min','mean'])

,max,min,mean
symbol,,,
000020,9490,9130,9331.0
000040,540,512,522.9
000050,10930,10820,10897.0
000070,75900,71300,72770.0
000080,25750,24450,25265.0
...,...,...,...
950170,4725,4500,4593.5
950190,13670,12780,13099.0
950200,7690,6980,7299.0


In [43]:
left = stock[stock.stock_date == pd.to_datetime('2023-02-09')]

/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [44]:
right = stock[stock.stock_date == pd.to_datetime('2023-02-22')]

In [45]:
right

,symbol,stock_date,close,before,open,high,low,vol,money
9,000020,2023-02-22,9130,150,9210,9210,9120,60428,551707640
19,000040,2023-02-22,515,8,523,523,512,166625,85811875
29,000050,2023-02-22,10930,0,10930,11070,10820,4914,53710020
39,000070,2023-02-22,75900,1500,73700,76200,73600,11611,881274900
49,000080,2023-02-22,24450,700,24950,24950,24400,261454,6392550300
...,...,...,...,...,...,...,...,...,...
25714,950170,2023-02-22,4515,30,4450,4730,4425,64006,288987090
25724,950190,2023-02-22,12780,190,12810,12900,12690,19083,243880740
25734,950200,2023-02-22,7260,210,7360,7520,7220,66344,481657440
25744,950210,2023-02-22,10010,200,10000,10290,9790,114419,1145334190


In [46]:
left = stock[stock.stock_date == pd.to_datetime('2023-02-09')]
right = stock[stock.stock_date == pd.to_datetime('2023-02-22')]
rt = pd.merge(left[['symbol', 'close']], right[['symbol', 'close']], right_on='symbol',
         left_on='symbol', how='inner')


/Users/happyeon/opt/anaconda3/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [48]:
rt

,symbol,close_x,close_y
0,000020,9490,9130
1,000040,540,515
2,000050,10930,10930
3,000070,71700,75900
4,000080,25750,24450
...,...,...,...
2569,950170,4570,4515
2570,950190,13670,12780
2571,950200,7530,7260
2572,950210,9630,10010


In [50]:
rt['수익율'] = (rt['close_y'] - rt['close_x']) / rt['close_x'] * 100

In [52]:
pd.merge(rt.sort_values(by=['수익율']), df[['ISU_NM', 'ISU_SRT_CD']], left_on='symbol', 
         right_on = 'ISU_SRT_CD', how='inner')

,symbol,close_x,close_y,수익율,ISU_NM,ISU_SRT_CD
0,083470,6200,2985,-51.854839,이엠앤아이,083470
1,084180,1295,780,-39.768340,수성샐바시온,084180
2,101000,553,345,-37.613020,상상인인더스트리,101000
3,291230,9810,6170,-37.104995,엔피,291230
4,058450,1272,840,-33.962264,일야,058450
...,...,...,...,...,...,...
2569,086900,133600,207500,55.314371,메디톡스,086900
2570,108860,14500,23200,60.000000,셀바스에이아이,108860
2571,208370,2435,4295,76.386037,셀바스헬스케어,208370
2572,229500,10400,19490,87.403846,노브메타파마,229500


In [56]:
pd.DataFrame(data=[[1,2,3],[4,5,6]], columns=['num1','num2','num3'])

,num1,num2,num3
0,1,2,3
1,4,5,6


In [60]:
tmp = pd.DataFrame([{'a':1 , 'b':2}, {'a':3, 'b':4}])

In [61]:
tmp2 = tmp